In [5]:
import pandas as pd
import numpy as np


In [6]:
import ast  
movies = pd.read_csv('movies_1.0.csv')


#sets movieId as the key
movies.set_index('movieId', inplace=True)

#converts genre to list
movies['genres'] = movies['genres'].apply(ast.literal_eval)


lf_matrix = pd.read_csv('movie_latent_factors_1.0.csv', header=None).values.tolist()

In [7]:
decades = [1980, 2000]
genres = ['Action', 'Fantasy']

In [8]:
#gets subset of movies captured by filters
genres_filter = (len(genres) == 0) | (movies['genres'].apply(lambda x: any(genre in x for genre in genres)))
decades_filter = (len(decades) == 0) | (movies['year'].apply(lambda x: any(x >= decade and x < decade + 10 for decade in decades)))

filtered_movies = movies[genres_filter & decades_filter]


In [9]:
from numpy.linalg import norm
def cosine_sim(vec1, vec2):
    sim = 0
    if norm(vec1) != 0 and norm(vec2) != 0:
        sim = np.dot(vec1, vec2) / (norm(vec1) * norm(vec2))
    return sim

#movie used to get recommendation
movie1 = 'Chef (2014)'
movie2 = 'Coco (2017)'

# search = movies[movies['title'].str.contains(movie, case=False, na=False)]

movie_index1 = movies.loc[movies['title'] == movie1].index[0]
movie_index2 = movies.loc[movies['title'] == movie2].index[0]

movie_vec1 = np.array(lf_matrix[movie_index1])
movie_vec2 = np.array(lf_matrix[movie_index2])

avg_vec = movie_vec1 + movie_vec2 / 2

# goes through filtered movies to get highest consine similarity
sim_list = []
movie_list = []
for id in filtered_movies.index:
    movie_vec = lf_matrix[id]
    sim = cosine_sim(avg_vec, movie_vec)
    sim_list.append(sim)
    movie_list.append(id)

combined_lists = list(zip(sim_list, movie_list))
sorted_combined_lists = sorted(combined_lists, key=lambda x: x[0], reverse = True)
sim_list, movie_list = zip(*sorted_combined_lists)

print("The 5 highest ranked for the query are: ")

for i in range(5):
    print(sim_list[i], '|', movies.loc[movie_list[i]]['title'])


The 5 highest ranked for the query are: 
0.9497888042987518 | Mulan (2009)
0.9469898874446199 | Wow! A Talking Fish! (1983)
0.9333195152618642 | Sky Fighters (Les Chevaliers Du Ciel) (2005)
0.9136343426932692 | Christmas Carol, A (2009)
0.9006689697330236 | High Security Vacation (2009)
